In [1]:
# testing out the logits thing here
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('LLM_IPD')

In [2]:
from utils import *

In [3]:
print_all_models()

google/gemma-7b
aegunal/FT_IPD_gemma7b
mistralai/Mistral-7B-v0.1
aegunal/FT_IPD_mistral7b
meta-llama/Llama-2-7b-hf
aegunal/FT_IPD_llama7b
tiiuae/falcon-7b
gt
t4t
cooperate
defect


In [3]:
# initialize models (want to initialize so they're fresh for each game)
model1 = PDModel(model_name_in="google/gemma-7b",
                player_id_in="1")
model2 = PDModel(model_name_in="meta-llama/Llama-2-7b-hf",
                player_id_in="2")

loading configuration file config.json from cache at /home/gunala/.cache/huggingface/hub/models--google--gemma-7b/snapshots/359f554e6b7dcfe8026a8705a20ff166141a5adf/config.json
Model config GemmaConfig {
  "_name_or_path": "google/gemma-7b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 24576,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 256000
}

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in 8-bit or 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing GemmaForCausalLM.

All the weights of GemmaForCausalLM were initialized from the model checkpoint at google/gemma-7b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GemmaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/gunala/.cache/huggingface/hub/models--google--gemma-7b/snapshots/359f554e6b7dcfe8026a8705a20ff166141a5adf/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 1,
  "pad_token_id": 0
}

loading file tokenizer.model from cache at /home/gunala/.cache/huggingface/hub/models--google--gemma-7b/snapshots/359f554e6b7dcfe8026a8705a20ff166141a5adf/tokenizer.model
loading file tokenizer.json from cache at /home/gunala/.cache/huggingface/hub/models--google--gemma-7b/snapshots/359f554e6b7dcfe8026a8705a20ff166141a5adf/tokenizer.json
loading file added_tokens.j

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Llama-2-7b-hf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/gunala/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.model from cache at /home/gunala/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/tokenizer.model
loading file tokenizer.json from cache at /home/gunala/.cache/huggingface/hub/models--me

In [4]:
num_rounds = 20
num_games = 10
sv_game_history = {}

for game_id in range(num_games):
    # game_history = ""
    sv_game_history[game_id] = {}
    logger.info("Game: " + str(game_id))
    for round_id in range(num_rounds):
        logger.info("Round: " + str(round_id))

        model1_info = model1.next_action_hf()
        model2_info = model2.next_action_hf()#next_action_classic()

        logger.info("model 1 action: " + model1_info['raw_logits']['action'])
        logger.info("model 2 action: " + model2_info['raw_logits']['action'])

        # save game hist
        sv_game_history[game_id][round_id] = {'p1':model1_info,'p2':model2_info}

        # update game histories for individual models
        model1.add_to_hist(round_id, model1_info['raw_logits']['action'], model2_info['raw_logits']['action'])
        model2.add_to_hist(round_id, model2_info['raw_logits']['action'], model1_info['raw_logits']['action'])

    model1.reset()
    model2.reset()

INFO:LLM_IPD:Game: 0
INFO:LLM_IPD:Round: 0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/gunala/miniconda3/envs/QLORAFTEnv/lib/python3.11/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided

model  1  output:  You are playing multiple rounds of prisoner's dilemma.You are Player 1. Do you choose to 'Cooperate' or 'Defect'? Game history: . Player 1:  Cooper


INFO:LLM_IPD:model 1 action: Cooperate
INFO:LLM_IPD:model 2 action: Cooperate
INFO:LLM_IPD:Round: 1


model  2  output:  You are playing multiple rounds of prisoner's dilemma.You are Player 2. Do you choose to 'Cooperate' or 'Defect'? Game history: . Player 2:   Player


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

model  2  output:  You are playing multiple rounds of prisoner's dilemma.You are Player 2. Do you choose to 'Cooperate' or 'Defect'? Game history: . Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2:   Cooper


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


model  1  output:  You are playing multiple rounds of prisoner's dilemma.You are Player 1. Do you choose to 'Cooperate' or 'Defect'? Game history: . Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1: Cooperate, Player 2: Cooperate. Player 1:  Cooper


INFO:LLM_IPD:model 1 action: Cooperate
INFO:LLM_IPD:model 2 action: Cooperate


model  2  output:  You are playing multiple rounds of prisoner's dilemma.You are Player 2. Do you choose to 'Cooperate' or 'Defect'? Game history: . Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2: Cooperate, Player 1: Cooperate. Player 2:   Cooper


In [5]:
import json
output_dirname = "/home/gunala/LLMDialGen/games/scripts/prompt4/"
output_fname = "gemma7b_VS_mistral7b"
with open(output_dirname + output_fname,'w+') as outf:
    json.dump(sv_game_history,outf)